# Reprompt SDK
This notebook demonstrates how to use the Reprompt SDK to list all batches and identify failed ones.

In [1]:
# Install reprompt if not already installed
!pip install -e ../

Obtaining file:///Users/lukasmartinelli/Repositories/reprompt
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for reprompt (pyproject.toml) ... done
  Created wheel for reprompt: filename=reprompt-0.0.9.0-py3-none-any.whl size=4033 sha256=fa748813cd3e2af6b618b8b217a8c73c8df1aac1fb3ceba7312e07389824290b
  Stored in directory: /private/var/folders/hx/5rvtdl9d499c5z1_fb75nqx40000gn/T/pip-ephem-wheel-cache-ocjk2bu4/wheels/7b/6c/c5/65a98a3ae06ed35383aaa03db9ec410458073d8ef109493b4d
Successfully built reprompt
  Attempting uninstall: reprompt
    Found existing installation: reprompt 0.0.9.0
    Uninstalling reprompt-0.0.9.0:
      Successfully uninstalled reprompt-0.0.9.0

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


Create a RepromptClient instance with your credentials.

In [2]:
import os
from reprompt import RepromptClient

api_key = os.getenv("REPROMPT_API_KEY")
org_slug = os.getenv("REPROMPT_ORG_SLUG")

client = RepromptClient(api_key=api_key, org_slug=org_slug)

In [ ]:
e2e_jobs_df = client.batches.list_batches(query="E2E").get_jobs().to_df()
e2e_jobs_df

In [4]:
e2e_jobs_df.to_csv("e2e_jobs_df.csv")

In [ ]:
import logging
logging.getLogger("httpx").setLevel(logging.WARNING)


batches = client.batches.list_batches()
for b in batches:
    #print(b)
    if b.status_counts['failed'] > 0:
        for job in client.jobs.get_jobs_by_batch_id(b.id):
            if job.status == 'failed':
                print("Found failed job", job.place_id, job.inputs.name)

In [ ]:
batches_fully_enumerated = list(client.batches.list_batches(query="E2E", page_size=100))
for b in batches_fully_enumerated[:3]:
    batch_details = client.batches.get_batch(b.id)
    # batch_details is a dict, not an object
    if 'jobs' in batch_details and 'completed' in batch_details['jobs']:
        for place_id in batch_details['jobs']['completed'][:2]:
            job_details = client.jobs.get_job(place_id)
            print(job_details)

In [ ]:
client.batches.list_batches(query="E2E", page_size=100).to_df()